<a href="https://colab.research.google.com/github/WisnuPrayogo096/analisisbigdata_1/blob/main/Modul5_Sem7_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import library
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import wordnet
import random
import string
import tensorflow as tf

In [ ]:
# download package
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/My Drive/Dataset_Sem7/dataset_praktikum/cleaned_reviews.csv")

In [ ]:
# Tampilkan 5 data teratas
print("5 Data Teratas:")
df.head()

5 Data Teratas:


,sentiments,cleaned_review,cleaned_review_length,review_score
0,positive,i wish would have gotten one earlier love it a...,19,5.0
1,neutral,i ve learned this lesson again open the packag...,88,1.0
2,neutral,it is so slow and lags find better option,9,2.0
3,neutral,roller ball stopped working within months of m...,12,1.0
4,neutral,i like the color and size but it few days out ...,21,1.0


In [ ]:
# Tampilkan 5 data terbawah
print("\n5 Data Terbawah:")
df.tail()


5 Data Terbawah:


,sentiments,cleaned_review,cleaned_review_length,review_score
17335,positive,i love this speaker and love can take it anywh...,30,5.0
17336,positive,i use it in my house easy to connect and loud ...,13,4.0
17337,positive,the bass is good and the battery is amazing mu...,41,5.0
17338,positive,love it,2,5.0
17339,neutral,mono speaker,2,5.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17340 entries, 0 to 17339
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sentiments             17340 non-null  object 
 1   cleaned_review         17337 non-null  object 
 2   cleaned_review_length  17340 non-null  int64  
 3   review_score           17340 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 542.0+ KB


In [ ]:
jumlah_data, jumlah_kolom = df.shape
print("Jumlah data:", jumlah_data)

Jumlah data: 17340


In [ ]:
df.isnull().sum()

sentiments               0
cleaned_review           3
cleaned_review_length    0
review_score             0
dtype: int64

In [ ]:
df = df.dropna(subset=['cleaned_review'])

In [ ]:
df.isnull().sum()

sentiments               0
cleaned_review           0
cleaned_review_length    0
review_score             0
dtype: int64

In [ ]:
df.duplicated().sum()

1347

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.duplicated().sum()

0

In [ ]:
jumlah_data, jumlah_kolom = df.shape
print('Jumlah data:', jumlah_data)

Jumlah data: 15990


In [ ]:
# Fungsi untuk pra-pemrosesan teks
def text_preprocessing(teks):
    # Mengonversi teks menjadi huruf kecil
    teks = teks.lower()

    # Menghapus tanda baca
    teks = teks.translate(str.maketrans('', '', string.punctuation))

    # Tokenisasi bisa ditambahkan jika diperlukan

    return teks

# Melakukan pra-pemrosesan teks pada kolom 'cleaned_review'
df['cleaned_review'] = df['cleaned_review'].apply(text_preprocessing)

In [ ]:
# Mengunduh dataset kata-kata stop dalam bahasa Inggris dari NLTK.
nltk.download('stopwords')

# Mengimpor modul stopwords dari nltk.corpus untuk mengakses kata-kata stop.
from nltk.corpus import stopwords

# Membuat sebuah list kosong
stop_words = []

# Melakukan iterasi melalui setiap kata dalam daftar kata-kata stop dalam bahasa Inggris.
for w in stopwords.words('english'):
    # Menambahkan setiap kata stop ke dalam list.
    stop_words.append(w)

# Mencetak list yang berisi kata-kata stop dalam bahasa Inggris.
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def get_synonyms(word):
    # Membuat sebuah set kosong untuk menyimpan sinonim.
    synonyms = set()

    # Melakukan iterasi melalui setiap synset (kumpulan kata dengan makna yang mirip)
    # dalam WordNet yang terkait dengan kata yang diberikan.
    for syn in wordnet.synsets(word):
        # Melakukan iterasi melalui setiap lemma (kata turunan) dalam synset.
        for l in syn.lemmas():
            # Mendapatkan kata sinonim, mengganti karakter khusus seperti _ dan - dengan spasi, mengubah huruf menjadi huruf kecil
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm']) # membersihkan karakter yang bukan huruf.

            # Menambahkan sinonim ke dalam set sinonim.
            synonyms.add(synonym)

    # Menghapus kata awal (kata yang diberikan) dari set sinonim, jika ada.
    if word in synonyms:
        synonyms.remove(word)

    # Mengembalikan list dari sinonim-sinonim yang ditemukan.
    return list(synonyms)

In [ ]:
def synonym_replacement(words, n):
    # Memisahkan kata-kata dalam kalimat menjadi sebuah list dengan .split().
    words = words.split()

    # Membuat salinan list
    new_words = words.copy()
    # Membuat daftar kata-kata unik yang bukan merupakan kata-kata stop words.
    random_word_list = list(set([word for word in words if word not in stop_words]))
    # Mengacak urutan kata-kata unik.
    random.shuffle(random_word_list)

    # Inisialisasi variabel untuk menghitung berapa banyak kata yang sudah digantikan.
    num_replaced = 0

    # Melakukan iterasi melalui daftar kata-kata unik yang telah diacak.
    for random_word in random_word_list:
        # Mendapatkan sinonim-sinonim dari kata acak dengan fungsi get_synonyms() yang kita buat sebelumnya.
        synonyms = get_synonyms(random_word)

        # Jika ada setidaknya satu sinonim yang ditemukan.
        if len(synonyms) >= 1:
            # Memilih salah satu sinonim secara acak.
            synonym = random.choice(list(synonyms))

            # Mengganti kata acak dengan sinonimnya dalam new_words.
            new_words = [synonym if word == random_word else word for word in new_words]

            # Menambah jumlah kata yang sudah digantikan.
            num_replaced += 1

        # Jika jumlah kata yang sudah digantikan mencapai n, maka berhenti.
        if num_replaced >= n:
            break

    # Menggabungkan kata-kata dalam new_words menjadi sebuah kalimat.
    sentence = ' '.join(new_words)

    # Mengembalikan nilai dari fungsi menjadi sebuah kalimat dari kata-kata.
    return sentence

In [ ]:
def random_deletion(words, p):

    words = words.split()

    # Jika kalimat hanya terdiri dari satu kata, maka kalimat tersebut dikembalikan tanpa perubahan.
    if len(words) == 1:
        return words

    # Membuat list baru untuk kata-kata yang akan dipertahankan.
    new_words = []

    # Melakukan iterasi melalui kata-kata dalam kalimat.
    for word in words:
        # Menghasilkan angka acak antara 0 dan 1.
        r = random.uniform(0, 1)

        # Jika nilai acak r lebih besar dari p (probabilitas), maka kata tersebut akan dipertahankan dengan menambahkannya ke dalam list new_words.
        if r > p:
            new_words.append(word)

    # Jika tidak ada kata yang dipertahankan (semua kata dihapus), maka satu kata acak dipilih untuk tetap ada dalam kalimat.
    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    # Menggabungkan kata-kata yang dipertahankan dalam list menjadi sebuah kalimat.
    sentence = ' '.join(new_words)

    return sentence

In [ ]:
def swap_word(new_words):
    # Menghasilkan dua indeks acak dalam range dari 0 hingga panjang new_words - 1.
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0

    # Melakukan loop sampai indeks kedua tidak sama dengan indeks pertama.
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1

        # Jika counter melebihi 3, maka kembalikan new_words tanpa perubahan.
        if counter > 3:
            return new_words

    # Menukar kata di indeks pertama dengan kata di indeks kedua dalam new_words.
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]

    # Mengembalikan new_words setelah kata-kata ditukar.
    return new_words

In [ ]:
def random_swap(words, n):

    words = words.split()
    # Membuat salinan list kata-kata yang akan dimodifikasi.
    new_words = words.copy()

    # Melakukan pertukaran kata-kata sebanyak n kali.
    for _ in range(n):
        new_words = swap_word(new_words)  # Memanggil fungsi swap_word() yang sudah dibuat untuk menukar kata-kata.

    # Menggabungkan kata-kata yang sudah ditukar kembali menjadi sebuah kalimat.
    sentence = ' '.join(new_words)

    # Mengembalikan kalimat setelah semua pertukaran kata-kata selesai.
    return sentence

In [ ]:
df['cleaned_review']

0        i wish would have gotten one earlier love it a...
1        i ve learned this lesson again open the packag...
2                it is so slow and lags find better option
3        roller ball stopped working within months of m...
4        i like the color and size but it few days out ...
                               ...                        
17334    i bought it to keep it at work and everyone lo...
17335    i love this speaker and love can take it anywh...
17336    i use it in my house easy to connect and loud ...
17337    the bass is good and the battery is amazing mu...
17339                                         mono speaker
Name: cleaned_review, Length: 15990, dtype: object

In [ ]:
# Penerapan Easy Data Augmentation pada dataset tipe teks menggunakan Synonym Replacement, Random Swap dan Random Deletion
augmented_text = []
num_rows = len(df['cleaned_review'])

for n in range(min(1000, num_rows)):  # Memastikan loop tidak melampaui jumlah baris
    word = df['cleaned_review'].iloc[n]  # Menggunakan iloc untuk mengakses berdasarkan posisi

    synonym_replaced = synonym_replacement(word, 3)
    augmented_text.append(synonym_replaced)

    random_swapped = random_swap(word, 3)
    augmented_text.append(random_swapped)

    random_deleted = random_deletion(word, 0.2)
    augmented_text.append(random_deleted)

In [ ]:
# Menyimpan label bersama dengan teks asli
original_texts = df['cleaned_review'].tolist()
original_labels = df['sentiments'].tolist()

# Penerapan Easy Data Augmentation pada dataset tipe teks menggunakan Synonym Replacement, Random Swap, dan Random Deletion
augmented_text = []
augmented_labels = []
num_rows = len(df['cleaned_review'])

for n in range(min(1000, num_rows)):
    text = df['cleaned_review'].iloc[n]
    label = df['sentiments'].iloc[n]

    synonym_replaced = synonym_replacement(text, 3)
    augmented_text.append(synonym_replaced)
    augmented_labels.append(label)

    random_swapped = random_swap(text, 3)
    augmented_text.append(random_swapped)
    augmented_labels.append(label)

    random_deleted = random_deletion(text, 0.2)
    augmented_text.append(random_deleted)
    augmented_labels.append(label)

# Menggabungkan data asli dan augmentasi
augmented_df = pd.DataFrame({'cleaned_review': original_texts + augmented_text, 'sentiments': original_labels + augmented_labels})

In [ ]:
augmented_df

,cleaned_review,sentiments
0,i wish would have gotten one earlier love it a...,positive
1,i ve learned this lesson again open the packag...,neutral
2,it is so slow and lags find better option,neutral
3,roller ball stopped working within months of m...,neutral
4,i like the color and size but it few days out ...,neutral
...,...,...
18985,very work slow,neutral
18986,work very slow,neutral
18987,scroller is squeaky and make up stochasticity ...,neutral
18988,makes is scroller loose squeaky noise seems and,neutral


In [ ]:
from sklearn.model_selection import train_test_split

# Memisahkan fitur dan label
texts = augmented_df['cleaned_review'].tolist()
labels = augmented_df['sentiments'].tolist()

# Inisialisasi LabelEncoder
label_encoder = LabelEncoder()

# Mengonversi label menjadi format numerik
labels = label_encoder.fit_transform(labels)

# Membagi dataset menjadi data pelatihan dan pengujian
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=111)

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification

# Menggunakan tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Pastikan train_texts dan test_texts adalah list string
train_texts = [str(text) for text in train_texts]
test_texts = [str(text) for text in test_texts]

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

In [ ]:
# Membuat dataset TensorFlow
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

In [ ]:
# Memuat model BERT untuk klasifikasi teks (contoh: sentimen positive/negative/neutral)
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Proses pelatihan
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])
model.fit(train_dataset.batch(32), epochs=1)

  1/475 [..............................] - ETA: 15:48:03 - loss: 0.8903 - accuracy: 0.5312

KeyboardInterrupt: ignored

In [ ]:
# Evaluasi Model
loss, accuracy = model.evaluate(test_dataset.batch(16))
print("Akurasi:", accuracy)
print("Loss:", loss)

In [ ]:
# Prediksi Model
predictions = model.predict(test_dataset.batch(16))
predicted_classes = tf.argmax(predictions.logits, axis=1)

In [ ]:
# Menampilkan laporan klasifikasi
target_names = ['Negative', 'Positive', 'Neutral']
print(classification_report(test_labels, predicted_classes, target_names=target_names))